kmp 2022

Based on **Lauwens & Downey "Think Julia: How to Think Like a Computer Scientist" 
https://benlauwens.github.io/ThinkJulia.jl/latest/book.html**

Resources:

Julia webpage https://julialang.org/ 

Julia documentation https://docs.julialang.org/en/v1/


## Chapter 06 -- Functions

https://benlauwens.github.io/ThinkJulia.jl/latest/book.html#chap06

Functions that do not return a result is called **void**: they might however have an **effect**, like printing a value or moving a turtle, but they return nothing. 

### Functions with Return Values

Calling a function that generates a return value is usually assigned to a variable or used as part of an expression.

```Julia
	e = exp(1.0)

	height = radius * sin(radians)

	function area(radius)
		a = π * radius^2
		return a
	end
```

The **return statement** is optional, and includes an expression, and means: “Return immediately from this function and use the following expression as a return value.” The expression can be arbitrarily complicated, so we can write this function more concisely:

```Julia
	function area(radius)
		return π * radius^2
	end
```

**The value returned by a function is the value of the last expression evaluated, which, by default, is the last expression in the body of the function definition.** On the other hand, using temporary variables like a and explicit return statements can make debugging easier.

Sometimes it is useful to have multiple return statements, one in each branch of a conditional:

```Julia
	function absvalue(x)
		if x < 0
			return -x
		else
			return x
		end
	end
```

Since these return statements are in an alternative conditional, only one runs. As soon as a return statement runs, the function terminates without executing any subsequent statements. 

Code that appears after a return statement, or any other place the flow of execution can never reach, is called **dead code**.

In a function, it is a good idea to ensure that every possible path through the program hits a return statement. For example:

```Julia
	function absvalue(x)   # incorrect
		if x < 0
			return -x
		elseif x > 0
			return x
		end
	end
```

This function is incorrect because if x happens to be 0, neither condition is true, and the function ends without hitting a return statement. If the flow of execution gets to the end of a function, the return value is nothing, which is not the absolute value of 0.

Julia provides a built-in function called **abs** that computes absolute values.

### Exercise 6-1

Write a compare function takes two values, x and y, and returns 1 if x > y, 0 if x == y, and -1 if x < y.

### Incremental Development

As you write larger functions, you might find yourself spending most of the time debugging.

To deal with increasingly complex programs, you might want to try a process called **incremental development**. The goal of incremental development is to avoid long debugging sessions by adding and testing only small amounts of code at a time.

As an example, suppose you want to find the distance between two points, given by the coordinates `(x1, y1)` and `(x2, y2)`. By the Pythagorean theorem, the distance is: 
`d = sqrt((x2−x1)^2 +(y2−y1)^2)`. 

The first step is to consider what a distance function should look like in Julia, what are the inputs (parameters) and what is the output (return value)? 

In this case, the inputs are two points, which you can represent using four numbers. The return value is the distance represented by a floating-point value. Immediately you can write an **outline** of the function:

```Julia
	function distance(x₁, y₁, x₂, y₂)
		0.0
	end
```

Obviously, this version does not compute distances; it always returns zero. But it is syntactically correct, and it runs, which means that you can test it before you make it more complicated. The subscript numbers are available in the **Unicode character encoding** (\_1<TAB>, \_2<TAB>, etc.). To test the new function, call it with sample arguments:

```Julia
	distance(1, 2, 4, 6)
```

When testing a function, it is useful to know the right answer. At this point we have confirmed that the function is syntactically correct, and we can start adding code to the body. A reasonable next step is to find the differences x₂ - x₁ and y₂ - y₁. The next version stores those values in temporary variables and prints them with the **@show macro**.

In [1]:
function distance(x₁, y₁, x₂, y₂)
    δx = x₂ - x₁	# \delta<TAB>
    δy = y₂ - y₁
    @show δx δy
    0.0
end

distance (generic function with 1 method)

In [2]:
distance(11, 12, 1, 2)

δx = -10
δy = -10


0.0

If the function is working, it should display δx = 3 and δy = 4. If so, we know that the function is getting the right arguments and performing the first computation correctly. Next we compute the sum of squares of δx and δy:

In [3]:
function distance(x₁, y₁, x₂, y₂)
    δx = x₂ - x₁
    δy = y₂ - y₁
    d² = δx^2 + δy^2
    @show d²
    0.0
end

distance (generic function with 1 method)

In [7]:
distance(11, 12, 11, 2)

10.0

Again, you would run the program at this stage and check the output (which should be 25). Superscript numbers are also available (\^2<TAB>). Finally, you can use sqrt to compute and return the result:

In [5]:
function distance(x₁, y₁, x₂, y₂)
    δx = x₂ - x₁
    δy = y₂ - y₁
    return sqrt(δx^2 + δy^2)
end

distance (generic function with 1 method)

In [10]:
distance(11, 12, 1, 2)

14.142135623730951

If that works correctly, you are done. Otherwise, you might want to print the value of sqrt(d²) before the return statement. The final version of the function does not display anything when it runs; it only returns a value. The print statements we wrote are useful for debugging, but once you get the function working, you should remove them. 

Code like that is called **scaffolding** because it is helpful for building the program but is not part of the final product. **Incremental development** can save you a lot of debugging time. The key aspects of the process are:

- Start with a working program and make small incremental changes. At any point, if there is an error, you should have a good idea where it is.

- Use variables to hold intermediate values so you can display and check them.

- Once the program is working, you might want to remove some of the scaffolding or consolidate multiple statements into compound expressions, but only if it does not make the program difficult to read.

### Exercise 6-2

Use incremental development to write a function called hypotenuse that returns the length of the hypotenuse of a right triangle given the lengths of the other two legs as arguments. Record each stage of the development process as you go.

### Composition

**You can call one function from within a function.** 

As an example, write a function that takes two points, the center of the circle and a point on the perimeter, and computes the area of the circle. Assume that the center point is stored in the variables xc and yc, and the perimeter point is in xp and yp. The first step is to find the radius of the circle, which is the distance between the two points. We just wrote a function, distance, that does that:

```Julia
	radius = distance(xc, yc, xp, yp)
```

The next step is to find the area of a circle with that radius; we  wrote that too:

```Julia
	result = area(radius)
```

**Encapsulating** these steps in a function, we get:

```Julia
	function circlearea(xc, yc, xp, yp)
		radius = distance(xc, yc, xp, yp)
		result = area(radius)
		return result
	end
```

The temporary variables radius and result are useful for development and debugging, but once the program is working, we can make it more concise by composing the function calls:

```Julia
	function circlearea(xc, yc, xp, yp)
		area(distance(xc, yc, xp, yp))
	end
```

### Boolean Functions

Functions that return booleans, called **predicates**, are often convenient for hiding complicated tests inside functions. For example:

```Julia
	function isdivisible(x, y)
		if x % y == 0
			return true
		else
			return false
		end
	end
```

It is common to give boolean functions names that sound like yes/no questions; _`isdivisible`_ returns either true or false to indicate whether x is divisible by y. 

The result of the **`==`** operator is a boolean, so we can write the function more concisely by returning it directly:

```Julia
	function isdivisible(x, y)
		return x % y == 0
	end
```

Boolean functions are often used in conditional statements:

```Julia
	if isdivisible(x, y)
		println("x is divisible by y")
	end
```

It might be tempting to write something like:

```Julia
	if isdivisible(x, y) == true 	# unnecessary
		println("x is divisible by y")
	end
```

But the extra comparison with true is unnecessary.

### Exercise 6-3
Write a function isbetween(x, y, z) that returns true if x ≤ y ≤ z or false otherwise.

### More Recursion

We have only covered a small subset of Julia, but this subset is a **complete programming language**, which means that anything that can be computed can be expressed in this language. 

Any program ever written could be rewritten using only the language features you have learned so far (actually, you would need a few commands to control devices like the mouse, disks, etc., but that’s all).

To give you an idea of what you can do with the tools you have learned so far, we will evaluate a few **recursively defined** mathematical functions. If you looked up the definition of the factorial function, denoted with the symbol **`!`**, you might get something like this:

`n! = 1 if n = 0, and = n(n−1)! if n > 0`

This definition says that the factorial of 0 is 1, and the factorial of any other value, n, is n multiplied by the factorial of n−1.

If you can write a recursive definition of something, you can write a Julia program to evaluate it. The first step is to decide what the parameters should be. In this case it should be clear that factorial takes an integer:

```Julia
	function fact(n) 
	end
```

If the argument happens to be 0, all we have to do is return 1:

```Julia
	function fact(n)
		if n == 0
			return 1
		end
	end
```

Otherwise, and this is the interesting part, we have to make a **recursive call** to find the factorial of n-1 and then multiply it by n:

```Julia
	function fact(n)
		if n == 0
			return 1
		else
			recurse = fact(n-1)
			result = n * recurse
			return result
		end
	end
```

In [9]:
function fact(n)    
    if n == 0
        return 1
    else
        recurse = fact(n-1)
        result = n * recurse
        return result
    end
end


m = BigInt(666)
result = fact(m)

1010632056840781493390822708129876451757582398324145411340420807357413802103697022989202806801491012040989802203557527039339704057130729302834542423840165856428740661530297972410682828699397176884342513509493787480774903493389255262878341761883261899426484944657161693131380311117619573051526423320389641805410816067607893067483259816815364609828668662748110385603657973284604842078094141556427708745345100598829488472505949071967727270911965060885209294340665506480226426083357901503097781140832497013738079112777615719116203317542199999489227144752667085796752482688850461263732284539176142365823973696764537603278769322286708855475069835681643710846140569769330065775414413083501043659572299454446517242824002140555140464296291001901438414675730552964914569269734038500764140551143642836128613304734147348086095123859660926788460671181469216252213374650499557831741950594827147225699896414088694251261045196672567495532228826719381606116974003112642111561332573503212960729711781993903877416394381

In [10]:
typeof(result)

BigInt

### One More Example

A famous recursively defined mathematical function is fibonacci, which has the following definition (see https://en.wikipedia.org/wiki/Fibonacci_number):

`fib(n)= 0 if n = 0, = 1 if n = 1 and = fib(n−1)+ fib(n−2) if n > 1`

Translated into Julia, it looks like this:

```Julia
	function fib(n)		# we will see more efficient implementations later on
		if n == 0
			return 0
		elseif n == 1
			return 1
		else
			return fib(n-1) + fib(n-2)
		end
	end
```

If you try to follow the flow of execution here, even for fairly small values of n, your head explodes. But according to the leap of faith, if you assume that the two recursive calls work correctly, then it is clear that you get the right result by adding them together.

In [3]:
function fact(n)

    if n == 0
        return 1
    else
        return n * fact(n-1)
    end
end

fact(1.5)

StackOverflowError: StackOverflowError:

### Checking Types

What happens if we call fact and give it 1.5 as an argument?

```Julia
	julia> fact(1.5)
		ERROR: StackOverflowError:
		Stacktrace:
 		[1] fact(::Float64) at ./REPL[3]:2
```

It looks like an infinite recursion. How can that be? The function has a **base case** when n == 0. But if n is not an integer, we can miss the base case and recurse forever. In the first recursive call, the value of n is 0.5. In the next, it is -0.5. From there, it gets more negative, but it will never be 0.

We have two choices. We can try to generalize the factorial function to work with floating-point numbers, or we can make fact check the type of its argument. The first option is called the **gamma function**. We will go for the second. We can use the built-in operator **isa** to verify the type of the argument. 

While we are at it, we can also make sure the argument is positive:

```Julia
	function fact(n)
		if !(n isa Int64)
			error("Factorial is only defined for integers.")
		elseif n < 0
			error("Factorial is not defined for negative integers.")
		elseif n == 0
			return 1
		else
			return n * fact(n-1)
		end
	end
```

The first base case handles non-integers; the second handles negative integers. In both cases, the program prints an error message and returns nothing to indicate that something went wrong:

```Julia
	julia> fact("fred")
		ERROR: Factorial is only defined for integers.

	julia> fact(-2)
		ERROR: Factorial is not defined for negative integers.
```

If we get past both checks, we know that n is positive or zero, so we can prove that the recursion terminates. This program demonstrates a pattern sometimes called a **guardian**. The first two conditionals act as guardians, protecting the code that follows from values that might cause an error. In **Catching Exceptions** we will see a more flexible alternative to printing an error message: **raising an exception**.

### Debugging

Breaking a large program into smaller functions creates natural checkpoints for debugging. If a function is not working, there are three possibilities to consider:

- There is something wrong with the arguments the function is getting; a **precondition** is violated.

- There is something wrong with the **function body**.

- There is something wrong with the return value, a **postcondition** is violated, or the way it is being used.

To rule out the first possibility, you can add a print statement at the beginning of the function and display the values of the parameters (and maybe their types). Or you can write code that checks the preconditions explicitly.

If the parameters look good, add a print statement before each return statement and display the return value. If possible, check the result by hand. Consider calling the function with values that make it easy to check the result (as in **Incremental Development**).

If the function seems to be working, look at the function call to make sure the return value is being used correctly (or used at all).

Adding print statements at the beginning and end of a function can help make the flow of execution more visible. For example, here is a version of fact with print statements:

```Julia
	function fact(n)
		space = " " ^ (4 * n)
		println(space, "factorial ", n)

		if n == 0
			println(space, "returning 1")
			return 1

		else
			recurse = fact(n-1)
			result = n * recurse
			println(space, "returning ", result)
			return result
		end
	end
```

space is a string of space characters that controls the indentation of the output:

```Julia
	julia> fact(4)
	                factorial 4
	            factorial 3
	        factorial 2
	    factorial 1
	factorial 0
	returning 1
	    returning 1
	        returning 2
	            returning 6
	                returning 24
	24
```
If you are confused about the flow of execution, this kind of output can be helpful. 

It takes some time to develop effective scaffolding, but a little bit of scaffolding can save a lot of debugging.

## Exercises

### Exercise 6-4

Draw a stack diagram for the following program. What does the program print?

```Julia
	function b(z)
		prod = a(z, z)
		println(z, " ", prod)
		prod
	end

	function a(x, y)
		x = x + 1
		x * y
	end

	function c(x, y, z)
		total = x + y + z
		square = b(total)^2
		square
	end

	x = 1
	y = x + 1
	println(c(x, y+3, x+y))
```

### Exercise 6-5

The Ackermann function, A(m,n), is defined as A(m,n) = n+1 if m = 0, = A(m−1,1) if m > 0 and n = 0, = A(m−1,A(m,n−1)) if m > 0 and n > 0. See https://en.wikipedia.org/wiki/Ackermann_function.

Write a function named ack that evaluates the Ackermann function. Use your function to evaluate ack(3, 4), which should be 125. What happens for larger values of m and n?


### Exercise 6-6

A palindrome is a word that is spelled the same backward and forward, like “noon” and “redivider”. Recursively, a word is a palindrome if the first and last letters are the same and the middle is a palindrome.

The following are functions that take a string argument and return the first, last, and middle letters:

```Julia
	function first(word)
		first = firstindex(word)
		word[first]
	end

	function last(word)
		last = lastindex(word)
		word[last]
	end

	function middle(word)
		first = firstindex(word)
		last = lastindex(word)
		word[nextind(word, first) : prevind(word, last)]
	end
```

We will see how they work in Strings.

Test these functions out. What happens if you call middle with a string with two letters? One letter? What about the empty string, which is written "" and contains no letters?

Write a function called ispalindrome that takes a string argument and returns true if it is a palindrome and false otherwise. Remember that you can use the built-in function length to check the length of a string.